<a href="https://colab.research.google.com/github/mkri/master/blob/master/agri_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building FastAi Model

In [12]:
#Printing thw working directory
import os
wd = os.getcwd()

In [2]:
wd

'/content'

In [3]:
# Setting The Home Directory for the project
home_dir = wd[:-7]

In [4]:
home_dir

'/'

In [5]:
#Listing the home directory and working directory contents
print('HOME DIRECTORY')
print(os.listdir(home_dir))
print('WORKING DIRECTORY')
print(os.listdir(wd))

HOME DIRECTORY
['srv', 'sys', 'root', 'var', 'usr', 'dev', 'proc', 'bin', 'sbin', 'run', 'mnt', 'home', 'boot', 'media', 'etc', 'opt', 'lib64', 'tmp', 'lib', 'content', '.dockerenv', 'datalab', 'tools', 'swift', 'tensorflow-1.15.2', 'lib32']
WORKING DIRECTORY
['.config', 'datafileNew.csv', 'sample_data']


## Loading The Datasets

In [142]:
import pandas as pd
import numpy as np
training_set = pd.read_csv(wd+'/datafileNew.csv')

In [143]:
training_set.head(5)

,Crop,Year,Month,Price,Location
0,Rice,2020,Jan,99.80,Mumbai
1,Wheat,2020,Feb,112.00,Pune
2,Coarse Cereals,2020,Mar,110.00,Chennai
3,Pulses,2020,Apr,134.00,Chennai
4,Vegetables,2020,May,103.45,Coimbatore


### Selecting Few Features

In [135]:
cols = ['Crop', 'Year', 'Month','Price', 'Location']

In [144]:
train_d = training_set[cols]

In [145]:
train_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Crop      179 non-null    object 
 1   Year      179 non-null    int64  
 2   Month     179 non-null    object 
 3   Price     179 non-null    float64
 4   Location  179 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 7.1+ KB


## Modeling With Fast.ai


In [146]:
from fastai.tabular import *
#This path will be used for saving and exporting the model
path = wd

In [147]:
#The target variable that we are trying to predict
dep_var = 'Price'

#The categorical variables 
cat_names = list(train_d.select_dtypes('object').columns)

#The continuous variables
cont_names =[] #No need to keep the Dependend variable

#Preprocessing steps for the fastai learner
procs = [FillMissing, Categorify, Normalize]

In [148]:
#Creating a validation set
val = TabularList.from_df(train_d.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

In [149]:
#Creating a trainig set
data = (TabularList.from_df(train_d, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(len(train_d) - int(len(train_d) * 0.2),len(train_d))))
                           .label_from_df(cols=dep_var)
                           .add_test(val)
                           .databunch())

In [150]:
data.show_batch()

Crop,Month,Location,target
Oilseeds,Jan,Mumbai,115.0
Fruits,Jan,Kochi,117.0
Fibers,Sep,Chennai,124.0
Fibers,Aug,Jaipur,109.0
"Eggs, Fish and Meat",Dec,Jaipur,137.0


### Initializing Neural Network

In [151]:
learn = tabular_learner(data, layers=[100,75,50,25], metrics= rmse)

### Training The Model

In [152]:
learn.fit(25, 1e-2)

epoch,train_loss,valid_loss,root_mean_squared_error,time
0,12280.515625,14012.603516,118.374847,00:00
1,12242.657227,13966.692383,118.180763,00:00
2,12253.839844,13900.385742,117.899887,00:00
3,12255.158203,13802.250000,117.482979,00:00
4,12227.377930,13689.191406,117.000816,00:00
5,12230.311523,13540.265625,116.362648,00:00
6,12203.735352,13385.567383,115.696014,00:00
7,12167.602539,13160.973633,114.721283,00:00
8,12138.395508,12913.643555,113.638214,00:00
9,12087.268555,12663.348633,112.531548,00:00


In [153]:
learn.show_results(ds_type=DatasetType.Train)

Crop,Month,Location,target,prediction
Vegetables,Feb,Kolkata,120.0,[18.35559]
Coarse Cereals,Oct,Mumbai,81.0,[-1.389244]
Rice,Nov,Chennai,105.0,[6.518528]
Sugarcane,Jan,Coimbatore,105.0,[10.249102]
Fruits,Jul,Hyderabad,81.0,[-4.11881]


In [154]:
learn.show_results(ds_type=DatasetType.Valid)

Crop,Month,Location,target,prediction
Oilseeds,Jan,Chennai,136.0,[7.200751]
Sugarcane,Feb,Kochi,129.0,[25.230421]
Fibers,Mar,Kolkata,115.0,[19.739527]
All Agriculture,Apr,Chennai,119.0,[23.62733]
Wheat,May,Coimbatore,124.0,[7.522519]


## Saving & Exporting The Model

In [155]:
learn.save('model',return_path=True)

PosixPath('/content/models/model.pth')

In [156]:
learn.export('model.pkl')